In [1]:
import numpy as np
import pandas as pd
import os
from datetime import date, datetime
from sqlalchemy import create_engine


In [2]:
# local connection information
db_user = os.environ.get('DB_USER')
db_pass = os.environ.get('DB_PASS')
engine = create_engine(f'mssql+pyodbc://{db_user}:{db_pass}' +
                       '@PSC-SQLProd/Campus6?' +
                       'driver=ODBC+Driver+13+for+SQL+Server')
connection = engine.connect()


In [3]:
sections_begin_year = '2011'


In [4]:
sql_str = "SELECT * FROM SECTIONS WHERE " + \
          "EVENT_SUB_TYPE NOT IN ('ADV') " + \
          f"AND ACADEMIC_YEAR >= '{sections_begin_year}' " + \
          "AND ACADEMIC_TERM IN ('FALL', 'SPRING', 'SUMMER') " + \
          "AND ACADEMIC_SESSION IN ('MAIN', 'CULN', 'EXT', 'FNRR', 'HEOP'," + \
          " 'SLAB', 'BLOCK A', 'BLOCK AB', 'BLOCK B') "
df_sections = pd.read_sql_query(sql_str, connection)


In [5]:
df = df_sections[['EVENT_ID', 'EVENT_SUB_TYPE', 'EVENT_MED_NAME',
                   'SECTION', 'CREDITS', 'MAX_PARTICIPANT',
                   'ACADEMIC_YEAR', 'ACADEMIC_TERM', 'ACADEMIC_SESSION',
                   'START_DATE', 'END_DATE', 'CIP_CODE',
                   'REVISION_DATE', 'REVISION_TIME',
                   ]]


print('ACADEMIC_TERM: ', df['ACADEMIC_TERM'].unique())

print('ACADEMIC_SESSION: ', df['ACADEMIC_SESSION'].unique())

In [6]:
print(df.shape)
df.head()

(5049, 14)


,EVENT_ID,EVENT_SUB_TYPE,EVENT_MED_NAME,SECTION,CREDITS,MAX_PARTICIPANT,ACADEMIC_YEAR,ACADEMIC_TERM,ACADEMIC_SESSION,START_DATE,END_DATE,CIP_CODE,REVISION_DATE,REVISION_TIME
0,ACC 101,LEC,Financial Accounting,01,3.0,35,2011,FALL,MAIN,2011-08-31,2011-12-16,,2013-08-19,1900-01-01 12:26:41.477
1,ACC 101,LEC,Financial Accounting,02,3.0,36,2011,FALL,MAIN,2011-08-31,2011-12-16,,2012-05-07,1900-01-01 13:59:40.743
2,ACC 101,LEC,Financial Accounting,03,3.0,30,2011,FALL,MAIN,2011-08-31,2011-12-16,None,2011-12-16,1900-01-01 17:22:31.393
3,ACC 201,LEC,Small Bus Acct,01,3.0,0,2011,FALL,MAIN,2011-08-31,2011-12-16,None,2011-09-28,1900-01-01 13:25:25.420
4,ACC 301,LEC,Small Business Acc,01,3.0,25,2011,FALL,MAIN,2011-08-31,2011-12-16,None,2012-01-19,1900-01-01 12:47:48.237


In [7]:
df = df[~(df['EVENT_ID'].str.contains('REG', case=False))]
df = df[~(df['EVENT_ID'].str.contains('STDY', case=False))]


In [8]:
print(df.shape)


(4855, 14)


In [9]:
df = df.rename(columns={'EVENT_MED_NAME': 'course_section_name',
                        'CREDITS': 'credit_hours',
                        'MAX_PARTICIPANT': 'maximum_enrollment_count',
                        'START_DATE': 'start_dt',
                        'END_DATE': 'end_dt',
                        'CIP_CODE': 'course_cip_code',
                        })


In [ ]:
df.loc[:, 'course_id'] = df.loc[:, 'EVENT_ID'].str.replace(' ', '')


In [ ]:
df.loc[:, 'course_section_id'] = (df['EVENT_ID'] + '.' +
                                  df['EVENT_SUB_TYPE'] + '.' +
                                  df['ACADEMIC_YEAR'] + '.' +
                                  df['ACADEMIC_TERM'].str.title() + '.' +
                                  df['SECTION']
                                  )
df.loc[:, 'integration_id'] = df.loc[:, 'course_section_id']


In [ ]:
term_id = (lambda c: (c['ACADEMIC_YEAR'] + '.' +
                      str(c['ACADEMIC_TERM']).title())
           if (c['ACADEMIC_SESSION'] == 'MAIN')
           else (c['ACADEMIC_YEAR'] + '.' +
                 str(c['ACADEMIC_TERM']).title() + '.' +
                 c['ACADEMIC_SESSION'])
           )
df.loc[:, 'term_id'] = df.apply(term_id, axis=1)


In [ ]:
df['AY'] = (pd.to_numeric(df['ACADEMIC_YEAR'], errors='coerce')
              .fillna(sections_begin_year).astype(np.int64))
cat_yr = (lambda c: c['AY'] if (c['ACADEMIC_TERM'] == 'FALL')
          else (c['AY'] - 1))
df.loc[:, 'catalog_year'] = df.apply(cat_yr, axis=1)


In [ ]:
crs_sect_delv = (lambda c: '03'
                 if str(c['SECTION'])[:2] == 'HY'
                 else ('02' if str(c['SECTION'])[:2] == 'ON'
                       else '01')
                 )
df.loc[:, 'course_section_delivery'] = df.apply(crs_sect_delv, axis=1)


In [ ]:
print(df.shape)
df.head()

In [ ]:
df[(df['SECTION'].str[:2]=='HY')]

In [ ]:
crs_integ_id = (lambda c: (c['EVENT_ID'] + '.' + str(c['catalog_year']))
                if (c['EVENT_SUB_TYPE'] == '')
                else (c['EVENT_ID'] + '.' + c['EVENT_SUB_TYPE'] + '.' +
                      str(c['catalog_year'])))
df.loc[:, 'course_integration_id'] = df.apply(crs_integ_id, axis=1)


In [ ]:
print(df.shape)
df.head()

In [ ]:
# read course_catalog.txt to find the correct catalog year
dfcat = pd.read_csv('../course_catalog/course_catalog.txt')
print(dfcat.shape)

In [ ]:
dfcat.head()

In [ ]:
dfcat = (dfcat[['course_id', 'integration_id']]
         .rename({'integration_id': 'cat_integ_id'}, axis='columns')
        )

In [ ]:
print(dfcat.shape)
dfcat.head()

In [ ]:
df = pd.merge(df, dfcat, on=['course_id'], how='left')

In [ ]:
print(df.shape)
df.head()

In [ ]:
df = (df.sort_values(['integration_id', 'course_integration_id'],
                     ascending=[True, True]))

df = df.loc[(df['course_integration_id'] >= df['cat_integ_id'])]

In [ ]:
print(df.shape)
df.head()

In [ ]:
df = (df.sort_values(['course_section_id', 'course_integration_id'],
                     ascending=[True, True]))
df[df.duplicated(['integration_id'])]


In [ ]:
print(df.shape)
df = (df.sort_values(['course_section_id', 'course_integration_id'],
                     ascending=[True, True])
      .drop_duplicates(['course_section_id'], keep='last')
      )
print(df.shape)

In [ ]:
df = df.loc[:, ['integration_id', 'course_section_name', 'course_section_id',
            'start_dt', 'end_dt', 'term_id', 'course_integration_id',
            'course_section_delivery', 'maximum_enrollment_count',
            'credit_hours',
            ]]

df = df.sort_values(['integration_id'])


In [ ]:
print(df.shape)
df.head()

In [ ]:
today = datetime.now().strftime('%Y%m%d')
fn_output = f'{today}_sections.txt'
df.to_csv(fn_output, index=False)
